#### Dicionário dos dados:

- id_municipio =ID Município 

- pib	=	Produto Interno Bruto a preços correntes

- impostos_liquidos	=	Impostos, líquidos de subsídios, sobre produtos a preços correntes

- va_agropecuaria	=	Valor adicionado bruto a preços correntes da agropecuária

- va_industria =	Valor adicionado bruto a preços correntes da indústria

- va_servicos	=	Valor adicionado bruto a preços correntes dos serviços, exclusive administração, defesa, educação e saúde públicas e seguridade social

- va_adespss =	Valor adicionado bruto a preços correntes da administração, defesa, educação e saúde públicas e seguridade social

OBS: O Valor adicionado representa a contribuição do setor para o Produto Interno Bruto (PIB) de um país, ou seja, o valor que ele cria na produção de bens e serviços, expresso na moeda corrente do período, sem a aplicação de qualquer fator de correção ou deflator de preços. Ele é calculado pela diferença entre o valor da produção agropecuária e o consumo intermediário (ou insumos) utilizado na sua fabricação.

**Fonte:** https://basedosdados.org/dataset/fcf025ca-8b19-4131-8e2d-5ddb12492347?table=fbbbe77e-d234-4113-8af5-98724a956943

## Métricas consolidadas Base PIB por UF

In [0]:
CREATE OR REPLACE TABLE data_lake_turing_.prata.metricas_pib_uf (
  WITH crescimento AS (
    SELECT
        sigla_uf,
        ano,
        pib,
        LAG(pib) OVER (PARTITION BY sigla_uf ORDER BY ano) AS pib_anterior,
        ROUND(
            (pib - LAG(pib) OVER (PARTITION BY sigla_uf ORDER BY ano)) 
            / LAG(pib) OVER (PARTITION BY sigla_uf ORDER BY ano) * 100, 2
        ) AS crescimento_percentual
    FROM data_lake_turing_.bronze.br_ibge_pib_uf
),
media_uf AS (
    SELECT
        sigla_uf,
        AVG(pib) AS avg_pib,
        AVG(crescimento_percentual) AS avg_crescimento
    FROM crescimento
    GROUP BY sigla_uf
)
SELECT
    NTILE(4) OVER (ORDER BY avg_pib) AS quartil,
    sigla_uf,
    avg_pib,
    avg_crescimento
FROM media_uf
ORDER BY quartil, avg_pib
)